# Aries Basic Controller - Issuer Alice

This notebook walks through how to issue a credential across a previously established connection. It is best run in parallel with the [Holder notebook](http://localhost:8889/notebooks/holder.ipynb) controlling the agent receiving the credential.

If unfamiliar with the protocol it is worth reading through the [aries-rfs](https://github.com/hyperledger/aries-rfcs/tree/master/features/0036-issue-credential)


In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)
    

## Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}
agent_controller.register_listeners([cred_listener], defaults=True)


## Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [4]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    


Results :  [{'connection_id': 'ba15ba75-51c1-4b6a-a771-c1c2891e179e', 'initiator': 'external', 'invitation_key': '3V2bVcNLhARUhiVvfHfxqWdc8p9Nt1rgzLuHckbUzfyR', 'created_at': '2020-07-09 15:42:49.132515Z', 'my_did': 'YZcXXhm22VW3LGpWy3iGtg', 'state': 'active', 'routing_state': 'none', 'their_label': 'Bob', 'updated_at': '2020-07-09 15:42:59.259713Z', 'their_did': '7KG1LK8j7YPBYCHGhBSSjc', 'request_id': '31debc08-1d95-4e32-8a92-79e9a51718e0', 'invitation_mode': 'once', 'accept': 'manual'}]
Connection : {'connection_id': 'ba15ba75-51c1-4b6a-a771-c1c2891e179e', 'initiator': 'external', 'invitation_key': '3V2bVcNLhARUhiVvfHfxqWdc8p9Nt1rgzLuHckbUzfyR', 'created_at': '2020-07-09 15:42:49.132515Z', 'my_did': 'YZcXXhm22VW3LGpWy3iGtg', 'state': 'active', 'routing_state': 'none', 'their_label': 'Bob', 'updated_at': '2020-07-09 15:42:59.259713Z', 'their_did': '7KG1LK8j7YPBYCHGhBSSjc', 'request_id': '31debc08-1d95-4e32-8a92-79e9a51718e0', 'invitation_mode': 'once', 'accept': 'manual'}
Active Conne

## Write a Schema to the Ledger

For more details see the [schema-api notebook](http://localhost:8888/notebooks/schema_api.ipynb)

In [5]:
# Define you schema name - must be unique on the ledger
schema_name = "open_mined_contributor"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["name", "skill"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)


PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1


## Write a Credential Definition to the Ledger

More details in the [definitions notebook](http://localhost:8888/notebooks/definitions_api.ipynb)

In [6]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default


**Note: You should be able to see both schema and definition transactions on the local network [here](http://localhost:9000)**

## Populate the Attribues to Issue to Bob

The schema defines two attributes: name and skill. Both must be populated with attribute values that Alice wishes to issue in a credential to Bob. To do this a list of objects must be created, each object containing the name of the attribute and it's value at a minimum. You can set the values to anything you wish.

TODO: Some additional fields such as mime-type can be defined.

In [7]:
credential_attributes = [
    {"name": "name", "value": "Bob"},
    {"name": "skill", "value": "researcher"}
]
print(credential_attributes)

[{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Send Credential

This sends a credential to a holder (Bob), and automates the rest of the protocol. This tutorial works best if you have initialised the [holder notebook](http://localhost:8889/notebooks/holder.ipynb) and registered the listener for the issue_credential webhook.

There are other ways to issue a credential that require multiple api calls.

**Arguments**
* connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
* schema_id: The id of the schema you wish to issue
* cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
* attributes: A list of attribute objects as defined above. Must match the schema attributes.
* comment (optional): Any string, defaults to ""
* auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
* trace (optional): Boolean, defaults to False. **Not entirely sure about this one, believe its for logging. Also when set to True it throws an error**

In [9]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")


{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'auto_remove': True, 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}, 'connection_id': 'ba15ba75-51c1-4b6a-a771-c1c2891e179e', 'trace': False, 'comment': '', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'schema_name': 'open_mined_contributor', 'schema_version': '0.0.1', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}
Credential exchange 937d7a76-aa6d-4728-8f3c-a23dee80f269, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 937d7a76-aa6d-4728-8f3c-a23dee80f269, role: issuer, state: offer_sent
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]


## Get Credential Exchange Records

The state of the protocol is stored in an object called a Credential Exchange. These can be retrieved from the agent either individually by ID or the full list.

Each record has a state, representing the stage of the protocol the record is at and a role. Either issuer or holder.

In [11]:
response = await agent_controller.issuer.get_records()
print(response['results'])

[{'credential_proposal_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': 'b6d92f87-a3cc-4b77-9516-d43d775b9ad9', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'schema_issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'comment': '', 'schema_version': '0.0.1', 'schema_name': 'open_mined_contributor', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]}}, 'updated_at': '2020-07-09 16:04:52.437427Z', 'auto_offer': False, 'auto_remove': True, 'trace': False, 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '462688020526099026688386717036386050277616972380

In [12]:
cred_record = await agent_controller.issuer.get_record_by_id(record_id)
state = cred_record['state']
role = cred_record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 937d7a76-aa6d-4728-8f3c-a23dee80f269, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 5def5a31-069f-498d-9053-095ae673d934, role: issuer, state: proposal_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]
Handle Credentials
Credential exchange 91e4e1e1-503e-49aa-917d-dda16adf953c, role: issuer, state: proposal_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]
Handle Credentials
Credential exchange 3b7e2b0c-f907-4dfd-b138-c526136925a3, role: issuer, state: proposal_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'software engineer'}]
Handle Credentials
Credential exchange 937d7a76-aa6d-4728-8f3c-a23dee80f269, role: issuer, state: request_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}]
Handle Credentials
Credential exchange 937d7a76-aa6d-4728-

## Remove Credential Exchange

It is possible to remove stored credential exchanges from an agent. Typically when you are not interested in the credential being offered, or the credential offer that you have sent has become stale (been ignored?).

In [15]:
response = await agent_controller.issuer.remove_record(record_id)
print(response)

ClientResponseError: 404, message='Not Found', url=URL('http://alice-agent:8021/issue-credential/records/22b59951-89c0-40d5-9d96-fa31093b2d25/remove')

ERROR:asyncio:Task exception was never retrieved
future: <Task finished coro=<run_in_terminal.<locals>.run() done, defined at /opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py:50> exception=UnsupportedOperation('fileno')>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/application/run_in_terminal.py", line 55, in run
    return func()
  File "/workspace/aries_basic_controller/utils.py", line 120, in <lambda>
    run_in_terminal(lambda: print_ext(*msg, color=color, **kwargs))
  File "/workspace/aries_basic_controller/utils.py", line 103, in print_ext
    print_formatted(FormattedText(msg), **kwargs)
  File "/workspace/aries_basic_controller/utils.py", line 83, in print_formatted
    prompt_toolkit.print_formatted_text(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/prompt_toolkit/shortcuts/utils.py", line 112, in print_formatted_text
    output = get_app_session().output
  File "/opt/cond

In [26]:
response = await agent_controller.issuer.get_records()
print(response)

{'results': [{'connection_id': 'ac53ec32-0c36-4202-a6b0-cbdff9b37ed1', 'credential_exchange_id': 'd318bc42-3bf8-4bdd-997f-3ec37b156ee4', 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'auto_offer': False, 'auto_remove': True, 'auto_issue': True, 'credential_offer': {'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:open_mined_contributor:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:10:default', 'key_correctness_proof': {'c': '79170176153350620028549950016135935654313248003471236926302455974413396051344', 'xz_cap': '1396235375325619727338321551222113480244211052014874840937160204552798576435859880925148151476722993078399547575807537695585488790036145257604049598327982178818544291205088081644473046348076996514883811028715602948870034170217016503327494499905844178648936888458598945201615966125928353232341132412886093303135418353600987329915719417804180125576821370613932982083996648803277155780075871308305170575550955969658720299874424052627072600097890088914615943336991

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

There are a few api calls not covered in this tutorial so far. Mostly around credential revocation.

In [5]:
response = await agent_controller.terminate()
print(response)

None
